In [54]:
import tensorflow as tf
import pandas as pd
import numpy as np
import nltk
import os

from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout, Concatenate
from tensorflow.python.ops import array_ops
from nltk import sent_tokenize, word_tokenize

from gcdc_data import load, load_pandas, Source, TrainOrTest

In [2]:
train_data = load_pandas(TrainOrTest.TRAIN, sources=[Source.CLINTON])
test_data = load_pandas(TrainOrTest.TEST, sources=[Source.CLINTON])

print(train_data['label'].unique())

train_data.head()

['2' '3' '1']


,text,label
0,Cheryl:\n\nAre we in a good place to begin pap...,2
1,"Our friend, General Joe Ballard owns The Raven...",2
2,Outstanding news! Miki Rakic called about 10 m...,3
3,Responding to separate emails from Uzra + Jeff...,1
4,Guy from Mexico is in NY and is cooperating. D...,1


In [3]:
t = tf.keras.preprocessing.text.Tokenizer(oov_token='unk')
t.fit_on_texts([
    word
    for doc in train_data['text']
    for sent in sent_tokenize(doc)
    for word in word_tokenize(sent)
])

vocab_size = len(t.word_index) + 1

vocab_size, t.word_index['unk']

(13163, 1)

In [4]:
EMBEDDING_DIM = 300

embeddings_index = {}
f = open(os.path.join('data', f'glove.6B.{EMBEDDING_DIM}d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found {} word vectors.'.format(len(embeddings_index)))

embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word, embeddings_index['unk'])
    embedding_matrix[i] = embedding_vector
        
print(embedding_matrix[72])

Found 400000 word vectors.
[-0.45414001 -0.55923003  0.72431999  0.74511999 -0.06304    -0.43191001
  0.1787      0.99759001 -0.51115    -0.79369998 -0.22916999 -0.47031999
  0.54519999  0.28490999 -0.11225     0.21269    -0.50792003 -0.64273
 -0.91689003 -1.00320005 -0.41808999 -0.40024    -0.21836001  0.025622
 -0.56033999 -0.64148998 -0.018587    0.23577     0.24417999  0.53899997
 -0.74356002 -0.14578     0.24029     0.013151   -0.43454999  0.59173
  0.41881001  0.47510999 -0.28112999 -0.10072    -0.98942    -0.49941
  1.0158      0.06466    -0.11672    -0.43849    -0.44154    -0.21893001
 -0.46342999 -0.57498997  0.052368   -0.79330999  0.24862     0.5359
 -0.60610998 -2.62479997 -0.4858     -0.059422    1.39530003  0.80589998
 -1.08729994  0.39109999 -0.22679999 -0.75388002  0.0095413  -0.11887
 -0.20058     1.07710004  0.53029001 -0.47058001  0.31349    -0.97333997
 -0.49098    -1.37979996 -0.62849998  0.3312     -0.19378    -0.39239001
 -1.2428     -0.0152      0.24099    -0.15

In [5]:
len(set(t.word_index.keys() - embeddings_index.keys()))

821

In [6]:
example_document = """Dear abby,

I'm writing to tell you you suck. Help me out of this mess.

Bye"""

In [7]:
def tokenize(text, tok=None):
    return [
        [
            tok.texts_to_sequences(nltk.word_tokenize(sent))
            if tok else
            nltk.word_tokenize(sent)
            for sent in nltk.sent_tokenize(para)
        ]
        for para in text.splitlines()
        if len(para) > 0
    ]

print(tokenize(example_document))

[[['Dear', 'abby', ',']], [['I', "'m", 'writing', 'to', 'tell', 'you', 'you', 'suck', '.'], ['Help', 'me', 'out', 'of', 'this', 'mess', '.']], [['Bye']]]


In [8]:
print(tokenize(example_document, t))

[[[[163], [1], []]], [[[8], [127], [792], [3], [340], [13], [13], [1], []], [[138], [47], [58], [5], [16], [5058], []]], [[[6344]]]]


In [9]:
train_data['tokenized'] = train_data['text'].map(lambda text: tokenize(text, t))
test_data['tokenized'] = test_data['text'].map(lambda text: tokenize(text, t))

train_data.head()

,text,label,tokenized
0,Cheryl:\n\nAre we in a good place to begin pap...,2,"[[[[122], []]], [[[23], [14], [7], [6], [100],..."
1,"Our friend, General Joe Ballard owns The Raven...",2,"[[[[30], [809], [], [301], [1211], [3403], [71..."
2,Outstanding news! Miki Rakic called about 10 m...,3,"[[[[1540], [294], []], [[5163], [7116], [203],..."
3,Responding to separate emails from Uzra + Jeff...,1,"[[[[2093], [3], [1348], [1455], [28], [5171], ..."
4,Guy from Mexico is in NY and is cooperating. D...,1,"[[[[1113], [28], [570], [11], [7], [657], [4],..."


In [10]:
MAX_DOC_LENGTH = 0
MAX_PARA_LENGTH = 0
MAX_SENT_LENGTH = 0

for doc in train_data['tokenized'].append(test_data['tokenized']):
    MAX_DOC_LENGTH = max(MAX_DOC_LENGTH, len(doc))
    for para in doc:
        MAX_PARA_LENGTH = max(MAX_PARA_LENGTH, len(para))
        for sent in para:
            MAX_SENT_LENGTH = max(MAX_SENT_LENGTH, len(sent))
            
MAX_DOC_LENGTH, MAX_PARA_LENGTH, MAX_SENT_LENGTH

(12, 19, 137)

In [11]:
def pad_to_dense(M, sent_len=MAX_SENT_LENGTH, para_len=MAX_PARA_LENGTH, doc_len=MAX_DOC_LENGTH):
    Z = np.zeros((len(M), doc_len, para_len, sent_len))
    for docidx, doc in enumerate(M):
        for paraidx, para in enumerate(doc):
            for sentidx, sent in enumerate(para):
                sentnp = np.hstack(np.array(sent))
                Z[docidx, paraidx, sentidx, :len(sentnp)] += sentnp
    return Z

print(train_data['tokenized'][0])
pad_to_dense(train_data['tokenized'][:1])

[[[[122], []]], [[[23], [14], [7], [6], [100], [234], [3], [620], [2593], [3], [469], [2946], [3397], [60], [112], [10], [7106], [7], [600], []]], [[[36], [41], [], [8], [31], [1450], [14], [306], [46], [3], [210], [10], [600], [1776], [4], [8], [17], [77], [2], [3398], [187], [3], [337], [2], [2593], []], [[55], [17], [18], [1659], [3], [48], [12], [1777], [172], [], [63], [17], [3399], [46], [3], [916], [807], [12], [2298], [], [4111], [], [7107], [], [284], [], [4], [16], [1777], [185], [1052], [17], [4112], [46], [178], [2], [245], [5], [2], [556], [10], [881], [516], [], [1923], []], [[16], [359], [31], [222], [47], [280], [172], [59], [41], [5], [2947], [140], [6], [4113], [1149], [12], [3400], [10], [769], [838], []]], [[[8], [66], [27], [2948], [10], [7108], [58], [3], [3400], [], [2], [2086], [14], [77], [2], [52], [44], [11], [3], [18], [191], [], [4], [14], [93], [27], [77], [2594], [], [4114], [58], [51], [140], [14], [1150], [], [27], [6], [7109], [7110], [808], [], [194],

array([[[[122.,   0.,   0., ...,   0.,   0.,   0.],
         [  0.,   0.,   0., ...,   0.,   0.,   0.],
         [  0.,   0.,   0., ...,   0.,   0.,   0.],
         ...,
         [  0.,   0.,   0., ...,   0.,   0.,   0.],
         [  0.,   0.,   0., ...,   0.,   0.,   0.],
         [  0.,   0.,   0., ...,   0.,   0.,   0.]],

        [[ 23.,  14.,   7., ...,   0.,   0.,   0.],
         [  0.,   0.,   0., ...,   0.,   0.,   0.],
         [  0.,   0.,   0., ...,   0.,   0.,   0.],
         ...,
         [  0.,   0.,   0., ...,   0.,   0.,   0.],
         [  0.,   0.,   0., ...,   0.,   0.,   0.],
         [  0.,   0.,   0., ...,   0.,   0.,   0.]],

        [[ 36.,  41.,   8., ...,   0.,   0.,   0.],
         [ 55.,  17.,  18., ...,   0.,   0.,   0.],
         [ 16., 359.,  31., ...,   0.,   0.,   0.],
         ...,
         [  0.,   0.,   0., ...,   0.,   0.,   0.],
         [  0.,   0.,   0., ...,   0.,   0.,   0.],
         [  0.,   0.,   0., ...,   0.,   0.,   0.]],

        ...,

  

In [12]:
def dense_mask(M, sent_len=MAX_SENT_LENGTH, para_len=MAX_PARA_LENGTH, doc_len=MAX_DOC_LENGTH):
    Z = np.zeros((len(M), doc_len, para_len, sent_len), dtype=bool)
    for docidx, doc in enumerate(M):
        for paraidx, para in enumerate(doc):
            for sentidx, sent in enumerate(para):
                for tokenidx, token in enumerate(sent):
                    Z[docidx, paraidx, sentidx, tokenidx] = True
    return Z

dense_mask(train_data['tokenized'][:1])

array([[[[ True,  True, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         ...,
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False]],

        [[ True,  True,  True, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         ...,
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False]],

        [[ True,  True,  True, ..., False, False, False],
         [ True,  True,  True, ..., False, False, False],
         [ True,  True,  True, ..., False, False, False],
         ...,
         [False, False, False, ..., False, False, False],
         [False, False, Fa

In [13]:
train_tensor = pad_to_dense(train_data['tokenized'])
test_tensor = pad_to_dense(test_data['tokenized'])

train_tensor.shape, test_tensor.shape

((1000, 12, 19, 137), (200, 12, 19, 137))

In [34]:
def categorical_labels(labels):
    eye = [
        [1.0, 0.0, 0.0],
        [0.0, 1.0, 0.0],
        [0.0, 0.0, 1.0]
    ]
    
    result = []
    for item in labels:
        result.append(eye[int(item) - 1])
        
    return np.array(result)

train_labels = categorical_labels(pd.to_numeric(train_data['label']))
test_labels = categorical_labels(pd.to_numeric(test_data['label']))

train_labels.shape, test_labels.shape

((1000, 3), (200, 3))

In [35]:
train_sent_mask = dense_mask(train_data['tokenized'])
test_sent_mask = dense_mask(test_data['tokenized'])

train_para_mask = np.apply_along_axis(any, 3, train_sent_mask)
test_para_mask = np.apply_along_axis(any, 3, test_sent_mask)

train_doc_mask = np.apply_along_axis(any, 2, train_para_mask)
test_doc_mask = np.apply_along_axis(any, 2, test_para_mask)

(train_sent_mask.shape, train_para_mask.shape, train_doc_mask.shape), (test_sent_mask.shape, test_para_mask.shape, test_doc_mask.shape)

(((1000, 12, 19, 137), (1000, 12, 19), (1000, 12)),
 ((200, 12, 19, 137), (200, 12, 19), (200, 12)))

In [36]:
BATCH_SIZE = 2

train_dataset = tf.data.Dataset.from_tensor_slices(
    ((train_tensor, train_sent_mask, train_para_mask, train_doc_mask), train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices(
    ((test_tensor, test_sent_mask, test_para_mask, test_doc_mask), test_labels))

train_dataset = train_dataset.shuffle(1000).batch(BATCH_SIZE)
test_dataset = test_dataset.shuffle(1000).batch(BATCH_SIZE)

(a, b, c, d), e = next(iter(train_dataset))
a.shape, b.shape, c.shape, d.shape, e.shape

(TensorShape([2, 12, 19, 137]),
 TensorShape([2, 12, 19, 137]),
 TensorShape([2, 12, 19]),
 TensorShape([2, 12]),
 TensorShape([2, 3]))

In [17]:
embedding = tf.keras.layers.Embedding(
    vocab_size,
    EMBEDDING_DIM,
    weights=[embedding_matrix],
    trainable=False)

embedded_example = embedding(train_tensor[:2])

print(embedded_example.shape)
embedded_example[0][0][0][0]

(2, 12, 19, 137, 100)


<tf.Tensor: id=63, shape=(100,), dtype=float32, numpy=
array([ 0.12616  , -0.2641   ,  0.071929 , -0.96047  , -0.086358 ,
       -0.032276 ,  0.33611  ,  0.55126  , -0.19631  , -0.66443  ,
       -0.16571  , -0.40401  , -0.13921  ,  0.44772  , -0.039273 ,
       -0.63014  , -0.2928   , -0.024373 , -0.81073  ,  0.70829  ,
       -0.47717  ,  0.39068  ,  0.44993  ,  0.25711  ,  0.68318  ,
        0.14003  , -0.013181 , -1.212    , -0.14414  ,  0.21759  ,
        0.30636  ,  0.7272   ,  0.82667  , -0.20531  , -0.68931  ,
       -0.047831 ,  0.3048   ,  0.20761  ,  0.33063  ,  0.33195  ,
       -0.23914  ,  0.046714 , -0.46688  ,  0.46208  ,  0.29071  ,
        0.60412  , -0.75673  , -0.34308  , -0.32161  , -0.17654  ,
        0.66982  ,  0.014476 , -0.12332  , -0.29709  ,  0.26196  ,
       -0.49916  , -0.65069  ,  0.3813   , -0.76894  , -0.2284   ,
       -0.25254  , -0.27246  ,  0.38411  ,  0.52052  ,  0.05651  ,
       -0.49209  ,  1.0191   ,  0.20061  , -0.33445  , -0.0094115,
       

In [55]:
# onions

class BahdanauAttentionLayer(tf.keras.layers.Layer):
    def __init__(self, units, **kwargs):
        super(BahdanauAttentionLayer, self).__init__(**kwargs)
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)
        
    def call(self, query, values):
        # (batch_size, ...) -> (batch_size, 1, ...)
        hidden_with_time_axis = tf.expand_dims(query, 1)
        
        # (batch_size, max_length, 1)
        scores = self.V(tf.nn.tanh(self.W1(values) + self.W2(hidden_with_time_axis)))
        
        # (batch_size, max_length, 1) normalized lulz
        attention_weights = tf.nn.softmax(scores, axis=1)
        
        context_vector = tf.reduce_sum(attention_weights * values, axis=1)
        
        return context_vector, attention_weights


class AttentiveSequenceEncoder(tf.keras.layers.Layer):
    def __init__(self, lstm_units, attention_units, **kwargs):
        super().__init__(**kwargs)
        self.lstm = Bidirectional(LSTM(lstm_units, recurrent_dropout=0.0001, return_state=True))
        self.concat = Concatenate()
        self.attention = BahdanauAttentionLayer(attention_units)
        
    def call(self, inputs, mask):
        sequence_encoded, forward_h, _, backward_h, _ = self.lstm(inputs, mask=mask)
        state_h = self.concat([forward_h, backward_h])
        output, attention_weights = self.attention(sequence_encoded, state_h)
        
        return output, attention_weights

In [46]:
BATCH_SIZE = 2  # keep it low


class DocModel(tf.keras.Model):
    def __init__(self, lstm_units, hidden_units, dropout, batch_size=BATCH_SIZE):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(
            vocab_size,
            EMBEDDING_DIM,
            weights=[embedding_matrix],
            trainable=False,
            mask_zero=True,
            input_shape=(MAX_DOC_LENGTH, MAX_PARA_LENGTH, MAX_SENT_LENGTH))
        
        self.sent_encoder = Bidirectional(LSTM(lstm_units, recurrent_dropout=0.0001))
        self.para_encoder = Bidirectional(LSTM(lstm_units, recurrent_dropout=0.0001))
        self.doc_encoder = Bidirectional(LSTM(lstm_units, recurrent_dropout=0.0001))
        
        self.hidden = tf.keras.layers.Dense(hidden_units, activation='tanh')
        self.dropout = tf.keras.layers.Dropout(dropout)
        self.classifier = tf.keras.layers.Dense(3, activation='sigmoid')
        
        self.dropout.build((BATCH_SIZE, hidden_units))
    
    def call(self, inputs, training=False):
        (inputs, sent_mask, para_mask, doc_mask) = inputs
        
        embedded = self.embedding(inputs)
        embedded = array_ops.reshape(
            embedded, (BATCH_SIZE * MAX_DOC_LENGTH * MAX_PARA_LENGTH, MAX_SENT_LENGTH, -1))
        sent_mask = array_ops.reshape(
            sent_mask, (BATCH_SIZE * MAX_DOC_LENGTH * MAX_PARA_LENGTH, MAX_SENT_LENGTH))

        sent_embedded = self.sent_encoder(embedded, mask=sent_mask)
        sent_embedded = array_ops.reshape(
            sent_embedded, (BATCH_SIZE * MAX_DOC_LENGTH, MAX_PARA_LENGTH, -1))
        para_mask = array_ops.reshape(
            para_mask, (BATCH_SIZE * MAX_DOC_LENGTH, MAX_PARA_LENGTH))
        
        para_embedded = self.para_encoder(sent_embedded, mask=para_mask)
        para_embedded = array_ops.reshape(
            para_embedded, (BATCH_SIZE, MAX_DOC_LENGTH, -1))
        
        x = self.doc_encoder(para_embedded, mask=doc_mask)
        x = self.hidden(x)
        x = self.dropout(x)
        
        return self.classifier(x)

doc_model = DocModel(150, 100, 0.5)
doc_model.compile(optimizer='adam', metrics=['accuracy'],
                  loss=tf.keras.losses.CategoricalCrossentropy())
doc_model(next(iter(train_dataset))[0])

doc_model.summary()


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Model: "doc_model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      multiple                  1316300   
_________________________________________________________________
bidirectional_3 (Bidirection multiple                  301200    
_________________________________________________________________
bidirectional_4 (Bidirection multiple                  541200    
_________________________

In [47]:
doc_model.fit(train_dataset, epochs=100)

Epoch 1/3
500/500 [==============================] - 119s 238ms/step - loss: 1.0472 - accuracy: 0.5040
Epoch 2/3
500/500 [==============================] - 111s 222ms/step - loss: 1.0698 - accuracy: 0.5040
Epoch 3/3
500/500 [==============================] - 113s 226ms/step - loss: 1.0361 - accuracy: 0.5080


In [48]:
doc_model.evaluate(test_dataset)

100/100 [==============================] - 9s 91ms/step - loss: 0.9966 - accuracy: 0.5550


[0.9965687346458435, 0.555]

In [57]:
class AttentiveDocModel(tf.keras.Model):
    def __init__(self, lstm_units, hidden_units, attention_units, dropout, batch_size=BATCH_SIZE):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(
            vocab_size,
            EMBEDDING_DIM,
            weights=[embedding_matrix],
            trainable=False,
            mask_zero=True,
            input_shape=(MAX_DOC_LENGTH, MAX_PARA_LENGTH, MAX_SENT_LENGTH))
        
        self.sent_encoder = AttentiveSequenceEncoder(lstm_units, attention_units)
        self.para_encoder = AttentiveSequenceEncoder(lstm_units, attention_units)
        self.doc_encoder = AttentiveSequenceEncoder(lstm_units, attention_units)
        
        self.hidden = tf.keras.layers.Dense(hidden_units, activation='tanh')
        self.dropout = tf.keras.layers.Dropout(dropout)
        self.classifier = tf.keras.layers.Dense(3, activation='sigmoid')
        
        self.dropout.build((BATCH_SIZE, hidden_units))
    
    def call(self, inputs, training=False):
        (inputs, sent_mask, para_mask, doc_mask) = inputs
        
        embedded = self.embedding(inputs)
        embedded = array_ops.reshape(
            embedded, (BATCH_SIZE * MAX_DOC_LENGTH * MAX_PARA_LENGTH, MAX_SENT_LENGTH, -1))
        sent_mask = array_ops.reshape(
            sent_mask, (BATCH_SIZE * MAX_DOC_LENGTH * MAX_PARA_LENGTH, MAX_SENT_LENGTH))

        sent_embedded, sent_weights = self.sent_encoder(embedded, mask=sent_mask)
        sent_embedded = array_ops.reshape(
            sent_embedded, (BATCH_SIZE * MAX_DOC_LENGTH, MAX_PARA_LENGTH, -1))
        para_mask = array_ops.reshape(
            para_mask, (BATCH_SIZE * MAX_DOC_LENGTH, MAX_PARA_LENGTH))
        
        para_embedded, para_weights = self.para_encoder(sent_embedded, mask=para_mask)
        para_embedded = array_ops.reshape(
            para_embedded, (BATCH_SIZE, MAX_DOC_LENGTH, -1))
        
        x, doc_weights = self.doc_encoder(para_embedded, mask=doc_mask)
        x = self.hidden(x)
        x = self.dropout(x)
        
        if not training:
            self.sent_weights = sent_weights
            self.para_weights = para_weights
            self.doc_weights = doc_weights
        
        return self.classifier(x)


attentive_doc_model = AttentiveDocModel(150, 100, 50, 0.5)
attentive_doc_model.compile(optimizer='adam', metrics=['accuracy'],
                            loss=tf.keras.losses.CategoricalCrossentropy())
attentive_doc_model(next(iter(train_dataset))[0])

attentive_doc_model.summary()


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Model: "attentive_doc_model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     multiple                  1316300   
_________________________________________________________________
attentive_sequence_encoder_9 multiple                  331351    
_________________________________________________________________
attentive_sequence_encoder_1 multiple                  571351    
_______________

In [58]:
attentive_doc_model.fit(train_dataset, epochs=100)

Epoch 1/3
500/500 [==============================] - 130s 260ms/step - loss: 1.0319 - accuracy: 0.5090
Epoch 2/3
 14/500 [..............................] - ETA: 2:04 - loss: 1.0596 - accuracy: 0.4615

KeyboardInterrupt: 

In [ ]:
attentive_doc_model.evaluate(test_dataset)

In [60]:
class SmallAttentiveDocModel(tf.keras.Model):
    def __init__(self, lstm_units, hidden_units, attention_units, dropout, batch_size=BATCH_SIZE):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(
            vocab_size,
            EMBEDDING_DIM,
            weights=[embedding_matrix],
            trainable=False,
            mask_zero=True,
            input_shape=(MAX_DOC_LENGTH, MAX_PARA_LENGTH, MAX_SENT_LENGTH))
        
        self.sent_encoder = AttentiveSequenceEncoder(lstm_units, attention_units)
        self.doc_encoder = AttentiveSequenceEncoder(lstm_units, attention_units)
        
        self.hidden = tf.keras.layers.Dense(hidden_units, activation='tanh')
        self.dropout = tf.keras.layers.Dropout(dropout)
        self.classifier = tf.keras.layers.Dense(3, activation='sigmoid')
        
        self.dropout.build((BATCH_SIZE, hidden_units))
    
    def call(self, inputs, training=False):
        (inputs, sent_mask, para_mask, doc_mask) = inputs
        
        embedded = self.embedding(inputs)
        embedded = array_ops.reshape(
            embedded, (BATCH_SIZE * MAX_DOC_LENGTH * MAX_PARA_LENGTH, MAX_SENT_LENGTH, -1))
        sent_mask = array_ops.reshape(
            sent_mask, (BATCH_SIZE * MAX_DOC_LENGTH * MAX_PARA_LENGTH, MAX_SENT_LENGTH))

        sent_embedded, sent_weights = self.sent_encoder(embedded, mask=sent_mask)
        sent_embedded = array_ops.reshape(
            sent_embedded, (BATCH_SIZE, MAX_DOC_LENGTH * MAX_PARA_LENGTH, -1))
        doc_mask = array_ops.reshape(
            para_mask, (BATCH_SIZE, MAX_DOC_LENGTH * MAX_PARA_LENGTH))
        
        x, doc_weights = self.doc_encoder(sent_embedded, mask=doc_mask)
        x = self.hidden(x)
        x = self.dropout(x)
        
        if not training:
            self.sent_weights = sent_weights
            self.doc_weights = doc_weights
        
        return self.classifier(x)


small_att_doc_model = SmallAttentiveDocModel(150, 100, 50, 0.5)
small_att_doc_model.compile(optimizer='adam', metrics=['accuracy'],
                            loss=tf.keras.losses.CategoricalCrossentropy())
small_att_doc_model(next(iter(train_dataset))[0])

small_att_doc_model.summary()


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Model: "small_attentive_doc_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     multiple                  1316300   
_________________________________________________________________
attentive_sequence_encoder_1 multiple                  331351    
_________________________________________________________________
attentive_sequence_encoder_1 multiple                  571351    
___________

In [61]:
small_att_doc_model.fit(train_dataset, epochs=100)

Epoch 1/3
     41/Unknown - 25s 619ms/step - loss: 1.1418 - accuracy: 0.5250

KeyboardInterrupt: 

In [ ]:
small_att_doc_model.evaluate(test_dataset)